### Suncal-Web Uncertainty Propagation

---

In [1]:
%matplotlib ipympl
import ipywidgets as widgets
import matplotlib.pyplot as plt

import suncalwidgets as sw
import suncal
suncal.plotting.plotstyle.update({'font.size': 9})

In [2]:
ucalc = suncal.UncertCalc(function='f=a+b')
ucalc.add_required_inputs()
ucalc.calculate()

funclist = sw.FunctionList()
inptlist = sw.InputList()
settings = sw.Settings()
output = sw.OutWidget(ucalc)

def on_funcnext(z):
    for i, func in enumerate(funclist.funclist):
        units = None if '[units]' in func.units.value else func.units.value
        try:
            ucalc.set_function(func.fexpr.value, idx=i, name=func.fname.value, outunits=units)
        except ValueError:
            output.view.layout.visibility = 'hidden'
            output.message('Invalid model equation')
            unctab.children = [funclist, output]
            unctab.set_title(1, 'Output')
            unctab.selected_index = 1
            return

    ucalc.add_required_inputs()
    for name in inptlist.get_names():
        if name not in ucalc.inputs.names:
            inptlist.rem_inpt(name)
    for name in ucalc.inputs.names:
        if name not in inptlist.get_names():
            inptlist.add_inpt(name)

    unctab.children = [funclist, inptlist]
    unctab.set_title(1, 'Measured Values')
    unctab.selected_index = 1

def on_measnext(z):
    inames = inptlist.get_names()
    for i, (inpt, name) in enumerate(zip(inptlist.inputs, inames)):
        ivar = ucalc.set_input(name=name, nom=inpt.nominal.value, units=inpt.units.value)
        for k, ucomp in enumerate(inpt.uncerts.children):
            uvar = ivar.add_comp(var=name, **ucomp.get_distargs())
        ivar.set_nom(inpt.nominal.value)

    settings.names = ucalc.inputs.names
    unctab.children = [funclist, inptlist, settings]
    unctab.set_title(2, 'Settings')
    unctab.selected_index = 2

def calculate(z):
    for cor in settings.get_corrs():
        ucalc.correlate_vars(*cor)

    if ucalc.model.check_circular():
        output.view.layout.visibility = 'hidden'
        output.message('Circular definition in measurement model')

    try:
        ucalc.model.check_dimensionality()
    except suncal.unitmgr.pint.PintError as e:
        output.view.layout.visibility = 'hidden'
        output.message('Units Error: \n\n' + str(e))

    try:
        ucalc.calculate()
    except (ValueError, IndexError) as e:
        output.view.layout.visibility = 'hidden'
        output.message('Error: \n\n' + str(e))
    except suncal.unitmgr.pint.PintError as e:
        output.view.layout.visibility = 'hidden'
        output.message('Units Error: \n\n' + str(e))
    else:
        output.view.layout.visibility = 'visible'
        output.update()

    unctab.children = [funclist, inptlist, settings, output]
    unctab.set_title(3, 'Output')
    unctab.selected_index = 3

funclist.btnnext.on_click(on_funcnext)
inptlist.btnnext.on_click(on_measnext)
settings.btnnext.on_click(calculate)

unctab = widgets.Tab(children=[funclist])
unctab.set_title(0, 'Model Functions')
unctab

---

Calculates combined uncertainty using the GUM and Monte Carlo (GUM-S1) methods


#### Tabs

- **Model Functions**: Enter one or more measurement models, with optional units
- **Measured Values**: Enter the measured value and uncertainty components for each variable in the model functions
- **Settings**: Enter number of Monte Carlo samples and random number seed, plus any correlation coefficients between input variables
- **Output**: Results of the calculation

---

#### Suncal

Suncal was developed by the Primary Standards Lab at Sandia National Laboratories. For more features and desktop version, see [https://sandiapsl.github.io](https://sandiapsl.github.io).

© 2019-2021 National Technology & Engineering Solutions of Sandia, LLC (NTESS).
Under the terms of Contract DE-NA0003525 with NTESS, the U.S. Government
retains certain rights in this software.